In [1]:
import os 
from bs4 import BeautifulSoup
import warnings
import logging


In [2]:
warnings.filterwarnings('error')

logger = logging.getLogger(__name__)

TRUTH_FILE_COLUMNS = ['id', 'gender', 'age_group','other_attribute']
# beware separator in the truth file is :::

In [3]:
#prsing 

In [4]:
with open("./en/1a0a6b860d3e89da11f1465090077569_en_30s_female.xml") as fp:
        xml = '\n'.join(fp.readlines())
        soup = BeautifulSoup(xml, 'html.parser')
        author = soup.author

In [5]:
for d in author.find_all('conversation'):
    text = d.get_text().strip()

In [6]:
text

'Personal finance advisors and columnists have worked hard to put the word out that credit unions are far better when it comes to fair customer dealings. When retail banking customers find their banking experience far from satisfactory at the large banks, the credit unions often appear as a breath of fresh air. So if the unions are so clean cut, what are they doing turning to the payday loan lender business?<br />;<br />;Does this mean that the credit unions are trying to clean up payday loans? Are they going to do it right?charging borrowers reasonable interest rates and so on? The traditional loan business isn\'t all that happy with the way credit unions are entering their turf. Especially when they believe that they aren\'t going to be much more reputable than the traditional businesses have ever been. They are just unhappy that unions are going to take business away from them by pretending to be better than the traditional lenders.<br />;<br />;Let\'s take a closer look. The Nation

In [7]:
type(text)

str

In [8]:
def parse_xml(filename):
    posts = []
    with open(filename) as fp:
        xml = '\n'.join(fp.readlines())
        soup = BeautifulSoup(xml, 'html.parser')
        author = soup.author
        author_attrs = dict(author.attrs)

        # the only child tags we have are conversation tags embedded in conversations tag
        for c in author.find_all('conversation'):
            text = c.get_text()
            posts.append(text)
            
        doc = dict()
        doc['id'] = filename.split("/")[-1][:32]
        doc['posts'] = posts
        doc['text'] = text
        doc.update({'attr.' + k: v for k, v in author_attrs.items()})
    return doc
            
    

In [9]:
doc = parse_xml("./en/1a0a6b860d3e89da11f1465090077569_en_30s_female.xml")

In [10]:
len(doc["posts"])

2

In [11]:
doc

{'attr.age_group': '30s',
 'attr.gender': 'female',
 'attr.lang': 'en',
 'id': '1a0a6b860d3e89da11f1465090077569',
 'posts': ['\n\n\t\t\tThis summer, some folks are wondering if they can afford to enjoy it as they have in the past. Ever since the recession, banks have been losing the respect of the public. Their reputation has been tarnished, and many are wondering where they can take their money for reliable, friendly and cost-effective service.<br />;<br />;Credit unions have been around for years, but people still wonder exactly how they work, and if they can get the same features and benefits of a big bank. The answer is yes. Most credit unions not only offer every product you can find at a bank, they usually offer it at substantial savings. Let\'s not forget better service, too.<br />;<br />;Banks are owned by shareholders, like any other company trying to make a profit. In order to make that profit, they need income. That comes from the high rates you pay on loans through them, o

In [12]:
# since we are getting problem with the tags in beautiful soup 
import re 
def remove_tags(text):
    cleanr = re.compile('<a.*>.*?</a>|<.*?>|\\n|\\t|;|\\>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

In [13]:
remove_tags(doc["posts"][0])


"This summer, some folks are wondering if they can afford to enjoy it as they have in the past. Ever since the recession, banks have been losing the respect of the public. Their reputation has been tarnished, and many are wondering where they can take their money for reliable, friendly and cost-effective service.Credit unions have been around for years, but people still wonder exactly how they work, and if they can get the same features and benefits of a big bank. The answer is yes. Most credit unions not only offer every product you can find at a bank, they usually offer it at substantial savings. Let's not forget better service, too.Banks are owned by shareholders, like any other company trying to make a profit. In order to make that profit, they need income. That comes from the high rates you pay on loans through them, or the fees you're assessed for using savings or checking accounts, or credit cards. Even deceptive totally-free offers usually come with unexpected fees later on, an

In [14]:
def parse_xml(filename):
    posts = []
    with open(filename) as fp:
        xml = '\n'.join(fp.readlines())
        soup = BeautifulSoup(xml, 'html.parser')
        author = soup.author
        author_attrs = dict(author.attrs)

        # the only child tags we have are conversation tags embedded in conversations tag
        for c in author.find_all('conversation'):
            text = remove_tags(c.get_text())
            posts.append(text)
            
        doc = dict()
        doc['id'] = filename.split("/")[-1][:32]
        doc['posts'] = posts
        doc['text'] = text
        doc.update({'attr.' + k: v for k, v in author_attrs.items()})
    return doc
            
  

In [15]:
# now we need to store all data into sqlite database
import sqlite3 as sql
conn = sql.connect("cleaned_english_tweets.db")

In [16]:
c = conn.cursor()
query = "CREATE TABLE tweets (\
user_id int(32),\
age int(2), \
gender varchar(1), \
post text \
);"
c.execute(query)

OperationalError: table tweets already exists